# Part 4

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
import cv2
import warnings
import numpy as np
from termcolor import colored
warnings.filterwarnings('ignore')

%matplotlib inline
from matplotlib import pyplot as plt

np.random.seed(202)

In [3]:
print(colored('\x1B[1mPath of dataset from drive.....','blue'))
#!unzip '/content/gdrive/MyDrive/Files/Part 4 Video.zip' -d '/content/gdrive/MyDrive/Files/Output/Video Part 4'
print(colored('\x1B[1mNot running again as Unziping is already done....','green'))

Path of dataset from drive.....
Not running again as Unziping is already done....


In [4]:
#Routine to fix 
def fixColor(image):
    return(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [5]:
print(colored('\x1B[1mIstalling IPython module','blue'))
#!pip install IPython
print(colored('\x1B[1mNot running again as installing is already done....','green'))

Istalling IPython module
Not running again as installing is already done....


In [6]:
print(colored('\x1B[1mUsing video 1 for detection','blue'))
print(colored('\x1B[1mTake a look at the input video','green'))
from IPython.core.display import Video
V1 = Video('/content/gdrive/MyDrive/Files/Output/Video Part 4/Video/video1.mp4',embed=True)
V1

Output hidden; open in https://colab.research.google.com to view.

In [7]:
print(colored('\x1B[1mTake a look at the input video','blue'))
print("Ecxtracting background from the video")
video_stream = cv2.VideoCapture('/content/gdrive/MyDrive/Files/Output/Video Part 4/Video/video1.mp4')

print("===Getting the count of the frame===")
frameIds = video_stream.get(cv2.CAP_PROP_FRAME_COUNT)
frameIds = int(frameIds)
# Store selected frames in an array

frames = []
for fid in range(frameIds):
    video_stream.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = video_stream.read()
    frames.append(frame)
    
# video_stream.release()

Take a look at the input video
Ecxtracting background from the video
===Getting the count of the frame===


In [8]:
writer = cv2.VideoWriter("/content/gdrive/MyDrive/Files/Output/output2_P4.mp4", cv2.VideoWriter_fourcc(*"MP4V"), 30,(640,480))

In [9]:
video_mask = []
from google.colab.patches import cv2_imshow

print("===Eliminating background and highlighting the objects===")

bcg_img = cv2.createBackgroundSubtractorMOG2(history = 500, varThreshold = 16, detectShadows = False)
for frame in frames:
  mask = bcg_img.apply(frame)
  video_mask.append(mask)

  #Improving the extraction by removing all the smaller elements and focus our attention on objects that are larger than a certain area.
  _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

  #Finding the contouring pixels of each object in the mask
  contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  #For each contour, we will the area of the contour and impute a bounding box on the objects where area is more than 500
  for cnt in contours:
        # Calculate area and remove small elements
    area = cv2.contourArea(cnt)
    if area > 500:
      x,y,w,h = cv2.boundingRect(cnt)
      # cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
      cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
  # plt.imshow(fixColor(frame))

  writer.write(cv2.resize(frame,(640,480)))

video_stream.release()
writer.release()

===Eliminating background and highlighting the objects===


In [10]:
pip install ffmpeg-python

In [11]:
import ffmpeg
import os
def compress_video(video_full_path, output_file_name, target_size):
    # Reference: https://en.wikipedia.org/wiki/Bit_rate#Encoding_bit_rate
    min_audio_bitrate = 32000
    max_audio_bitrate = 256000

    probe = ffmpeg.probe(video_full_path)
    
    # Video duration, in s.
    duration = float(probe['format']['duration'])
    print(duration)
    # Target total bitrate, in bps.
    target_total_bitrate = (target_size * 1024 * 8) / (1.073741824 * duration)

    video_bitrate = target_total_bitrate

    i = ffmpeg.input(video_full_path)
    ffmpeg.output(i, os.devnull,
                  **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 1, 'f': 'mp4'}
                  ).overwrite_output().run()
    ffmpeg.output(i, output_file_name,
                  **{'c:v': 'libx264', 'b:v': video_bitrate, 'pass': 1, 'f': 'mp4'}
                  ).overwrite_output().run()

compress_video('/content/gdrive/MyDrive/Files/Output/output2_P4.mp4', '/content/gdrive/MyDrive/Files/Output/output2_P4_compressed.mp4', 10 * 1000)

88.267


In [12]:
from IPython.core.display import HTML
from base64 import b64encode
video_path = '/content/gdrive/MyDrive/Files/Output/output2_P4_compressed.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=640 controls><source src={decoded_vid} type="video/mp4"></video>')

Output hidden; open in https://colab.research.google.com to view.